## 1. Introduction

### Dataset Description
- **Dataset Source**: REMS data collected by the Curiosity Rover. It contains **3,197 records** spanning multiple Martian years, with variables covering **temperature, pressure, UV radiation, and day length**.
- **Key Columns**:
  - `Ls`: Solar longitude representing Mars’ position in its orbit.
  - `sunrise`, `sunset`: Times for sunrise and sunset. 
  - `max_ground_temp`, `min_ground_temp`: Ground temperature extremes.
  - `max_air_temp`, `min_air_temp`: Air temperature extremes.
  - `avg_air_temp`, `avg_ground_temp`: Average air and ground temperatures.
  - `mars_month`: Martian month based on solar longitude (30  degrees per month)
  - `mars_year`: Martian year based on mission start (Initial year = 1)
  - `mars_season`: Martian season based on solar longitude. Curiosity is located in the southern hemisphere which means our seasons are inverted.
  - `day_length`: Length of the Martian day in minutes.
  - `mean_pressue`: Average atmospheric pressure for a given day.
  - `UV_Radiation`: UV index categories.
- Purpose: This dataset helps study Martian weather patterns and seasonal variations.

---

## 2. Data Overview

In [13]:
import os
import sys
import warnings
import joblib
import gc
from datetime import datetime

import pandas as pd
import numpy as np
from tqdm import tqdm  # Progress bar for long-running loops

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utilities import save_model_outputs

# Suppress non-critical warnings from dependencies
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Allow imports from project utilities folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Project directory configuration
DATA_PATH     = "../data/"
MODEL_PATH    = "../models/"
LOG_PATH_ROOT = "../logs/"

### 2.1 Load and Inspect Data

In [2]:
# Load processed datasets (indexed by Martian sol number)
# --- Unscaled data ---
X_unscaled_train = pd.read_csv(
    os.path.join(DATA_PATH, "processed/unscaled_train.csv"),
    index_col='sol_number'
)
X_unscaled_test  = pd.read_csv(
    os.path.join(DATA_PATH, "processed/unscaled_test.csv"),
    index_col='sol_number'
)

# Primary target series (ground temperature)
y_train = X_unscaled_train['avg_ground_temp']
y_test  = X_unscaled_test['avg_ground_temp']

In [3]:
X_unscaled_train.head()

,avg_air_temp,avg_ground_temp,mean_pressure,day_length,sin_lon,cos_lon,mars_season_spring,mars_season_summer,mars_season_winter,UV_Radiation_low,UV_Radiation_moderate,UV_Radiation_very_high
sol_number,,,,,,,,,,,,
1,-37.5,-45.5,739.0,712.0,0.500000,-0.866025,0,0,1,0,0,1
9,-37.5,-45.5,739.0,714.0,0.422618,-0.906308,0,0,1,0,0,1
10,-37.5,-45.5,739.0,714.0,0.422618,-0.906308,0,0,1,0,0,1
11,-37.0,-43.5,740.0,713.0,0.406737,-0.913545,0,0,1,0,0,1
12,-37.0,-47.0,741.0,713.0,0.406737,-0.913545,0,0,1,0,0,1


In [4]:
X_unscaled_test.head()

,avg_air_temp,avg_ground_temp,mean_pressure,day_length,sin_lon,cos_lon,mars_season_spring,mars_season_summer,mars_season_winter,UV_Radiation_low,UV_Radiation_moderate,UV_Radiation_very_high
sol_number,,,,,,,,,,,,
2695,-32.5,-40.0,712.0,715.0,0.325568,-0.945519,0,0,1,0,1,0
2696,-33.5,-36.5,713.0,715.0,0.325568,-0.945519,0,0,1,0,1,0
2697,-35.0,-35.5,716.0,716.0,0.309017,-0.951057,0,0,1,0,1,0
2698,-33.0,-39.5,715.0,716.0,0.292372,-0.956305,0,0,1,0,1,0
2699,-31.5,-37.0,715.0,715.0,0.292372,-0.956305,0,0,1,0,0,0


In [5]:
print(f"Unscaled training set shape : {X_unscaled_train.shape}")
print(f"Unscaled test set shape  : {X_unscaled_test.shape}")

Unscaled training set shape : (2557, 12)
Unscaled test set shape  : (640, 12)


In [6]:
print("Training Set Info")
print("---" * 30)
print(X_unscaled_train.info())

Training Set Info
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 2557 entries, 1 to 2694
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_air_temp            2557 non-null   float64
 1   avg_ground_temp         2557 non-null   float64
 2   mean_pressure           2557 non-null   float64
 3   day_length              2557 non-null   float64
 4   sin_lon                 2557 non-null   float64
 5   cos_lon                 2557 non-null   float64
 6   mars_season_spring      2557 non-null   int64  
 7   mars_season_summer      2557 non-null   int64  
 8   mars_season_winter      2557 non-null   int64  
 9   UV_Radiation_low        2557 non-null   int64  
 10  UV_Radiation_moderate   2557 non-null   int64  
 11  UV_Radiation_very_high  2557 non-null   int64  
dtypes: float64(6), int64(6)
memory usage: 259.

In [7]:
print("Test Set Info")
print("---" * 30)
print(X_unscaled_test.info())

Test Set Info
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 640 entries, 2695 to 3368
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   avg_air_temp            640 non-null    float64
 1   avg_ground_temp         640 non-null    float64
 2   mean_pressure           640 non-null    float64
 3   day_length              640 non-null    float64
 4   sin_lon                 640 non-null    float64
 5   cos_lon                 640 non-null    float64
 6   mars_season_spring      640 non-null    int64  
 7   mars_season_summer      640 non-null    int64  
 8   mars_season_winter      640 non-null    int64  
 9   UV_Radiation_low        640 non-null    int64  
 10  UV_Radiation_moderate   640 non-null    int64  
 11  UV_Radiation_very_high  640 non-null    int64  
dtypes: float64(6), int64(6)
memory usage: 65.0 K

---

## 3. Forecasting and Anomaly Detection

In [8]:
# Hyperparameter grid setup
# Non-seasonal (p, d, q) options: AR terms, differencing, MA terms
p = range(0, 3)   # AR order: 0–2
d = range(0, 2)   # Differencing: 0–1
q = range(0, 3)   # MA order: 0–2
pdq_grid = [
    (x, y, z)
    for x in p for y in d for z in q
    if (x, y, z) != (0, 0, 0)   # exclude degenerate model
]

# Seasonal parameters: (P, D, Q) at each period s
seasonal_periods   = [5,10,15,20,25,30,35] 
P = Q = [0, 1]                 # Seasonal AR/MA terms
D = [1]                        # Seasonal differencing
seasonal_pdq_grid = [
    (X, Y, Z)
    for X in P for Y in D for Z in Q
]

In [9]:
def search_best_seasonal_model(
    y_train: pd.Series,
    y_test: pd.Series,
    pdq_grid: list,
    seasonal_pdq_grid: list,
    seasonal_periods: list,
    exog_train: pd.DataFrame = None,
    exog_test: pd.DataFrame  = None,
    model_type: str          = "SARIMA",
    log_path: str            = None
):
    """
    Perform a serial grid search over SARIMA/SARIMAX hyperparameters.

    For each combination of (p,d,q) × (P,D,Q,s), fits a SARIMAX model,
    evaluates it on the hold-out series, and selects the best model by
    lowest RMSE (ties broken by AIC). Logs all AIC/MAE/RMSE results
    to a file and returns both the best-model metadata and a list of
    all tested configurations.

    Parameters:
        y_train (pd.Series): Training target series.
        y_test  (pd.Series): Test target series for out-of-sample evaluation.
        pdq_grid (list): List of non-seasonal (p,d,q) tuples.
        seasonal_pdq_grid (list): List of seasonal (P,D,Q) tuples (excl. period).
        seasonal_periods (list): List of seasonal period lengths to evaluate.
        exog_train (pd.DataFrame, optional): Exogenous regressors for training.
        exog_test  (pd.DataFrame, optional): Exogenous regressors for testing.
        model_type (str): Identifier used in logs and output filenames.
        log_path (str, optional): Path to log file. If None, defaults to
            {LOG_PATH_ROOT}/{model_type}_search.log.

    Returns:
        best_result (dict): {
            'seasonal_period', 'order', 'seasonal_order', 'aic',
            'mae', 'rmse', 'model', 'residuals'
        }
        all_models (list of dict): Metadata for every tested configuration.
    """
    # Ensure output directories exist
    os.makedirs(LOG_PATH_ROOT, exist_ok=True)
    os.makedirs(MODEL_PATH,    exist_ok=True)

    # Default log filename if none provided
    if log_path is None:
        log_path = os.path.join(
            LOG_PATH_ROOT,
            f"{model_type.lower()}_search.log"
        )

    # Initialize best‐model trackers
    best_rmse     = np.inf
    best_aic      = np.inf
    best_mae      = np.inf
    best_model    = None
    best_order    = None
    best_seasonal = None
    best_period   = None

    all_models = []

    # Compute total fits for tqdm bar
    total_fits = sum(
        1
        for s in seasonal_periods
        for order in pdq_grid
        for season in seasonal_pdq_grid
        if order != (0, 0, 0)
    )

    # Open log and progress bar
    with open(log_path, "w") as log_file, \
         tqdm(total=total_fits, desc=f"{model_type} grid search") as pbar:

        # Write header
        log_file.write(f"{model_type} Grid Search Log\n")
        log_file.write(f"{'='*60}\n")
        log_file.write(f"Started at: {datetime.now():%Y-%m-%d %H:%M:%S}\n")
        log_file.write(f"Seasonal periods: {seasonal_periods}\n\n")

        # Iterate through every hyperparameter combination
        for s in seasonal_periods:
            for order in pdq_grid:
                for season in seasonal_pdq_grid:

                    # Skip degenerate model
                    if order == (0, 0, 0):
                        pbar.update()
                        continue

                    try:
                        # Fit SARIMAX (or SARIMA if exog_train is None)
                        model = SARIMAX(
                            y_train,
                            exog=exog_train,
                            order=order,
                            seasonal_order=season + (s,)
                        )
                        result = model.fit(disp=False, maxiter=50)

                        # Evaluate: AIC (in-sample), then MAE/RMSE (out-of-sample)
                        aic   = result.aic
                        fc    = result.get_forecast(
                            steps=len(y_test),
                            exog=exog_test
                        )
                        y_pred = fc.predicted_mean
                        mae   = mean_absolute_error(y_test, y_pred)
                        rmse  = np.sqrt(mean_squared_error(y_test, y_pred))

                        # Record in full grid log
                        all_models.append({
                            "seasonal_period": s,
                            "order": order,
                            "seasonal_order": season,
                            "aic": aic,
                            "mae": mae,
                            "rmse": rmse
                        })
                        log_file.write(
                            f"{order} x {season+(s,)} → "
                            f"AIC={aic:.2f}, MAE={mae:.2f}, RMSE={rmse:.2f}\n"
                        )
                        log_file.write("-"*60 + "\n")
                        log_file.flush()

                        # Keep only the best model in memory
                        if (rmse < best_rmse) or \
                           (rmse == best_rmse and aic < best_aic):

                            if best_model is not None:
                                del best_model
                                gc.collect()

                            best_rmse     = rmse
                            best_aic      = aic
                            best_mae      = mae
                            best_model    = result
                            best_order    = order
                            best_seasonal = season
                            best_period   = s
                        else:
                            del result
                            gc.collect()

                    except Exception as err:
                        log_file.write(
                            f"FAILED {order} x {season+(s,)} → {err}\n"
                        )
                        log_file.write("-"*60 + "\n")
                        log_file.flush()

                    pbar.update()

    # Persist the best model to disk
    if best_model is not None:
        out_path = os.path.join(
            MODEL_PATH,
            f"best_{model_type.lower()}_model.pkl"
        )
        joblib.dump(best_model, out_path)

    # Extract out-of-sample residuals for downstream anomaly detection
    residuals = None
    if best_model is not None:
        fc = best_model.get_forecast(
            steps=len(y_test),
            exog=exog_test
        )
        y_pred    = fc.predicted_mean
        residuals = y_test - y_pred

    # Return best-model metadata and full grid results
    best_result = {
        "seasonal_period": best_period,
        "order": best_order,
        "seasonal_order": best_seasonal,
        "aic": best_aic,
        "mae": best_mae,
        "rmse": best_rmse,
        "model": best_model,
        "residuals": residuals
    }
    return best_result, all_models # Add y_pred

### 3.1 SARIMA

In [10]:
# Fit and evaluate SARIMA (no exogenous regressors)
best_sarima, all_sarima = search_best_seasonal_model(
    y_train       = y_train,
    y_test        = y_test,
    pdq_grid      = pdq_grid,
    seasonal_pdq_grid = seasonal_pdq_grid,
    seasonal_periods  = seasonal_periods,
    exog_train    = None,
    exog_test     = None,
    model_type    = "SARIMA",
    log_path      = os.path.join(LOG_PATH_ROOT, "sarima_search.log")
)

# Print SARIMA summary
print("Best SARIMA Configuration")
print("="*40)
print(f"Seasonal Period : {best_sarima['seasonal_period']}")
print(f"Order           : {best_sarima['order']}")
print(f"Seasonal Order  : {best_sarima['seasonal_order']} + ({best_sarima['seasonal_period']})")
print(f"AIC             : {best_sarima['aic']:.2f}")
print(f"MAE             : {best_sarima['mae']:.2f}")
print(f"RMSE            : {best_sarima['rmse']:.2f}")
print("="*40, "\n")

SARIMA grid search: 100%|█████████████████████| 476/476 [39:50<00:00,  5.02s/it]


Best SARIMA Configuration
Seasonal Period : 25
Order           : (2, 0, 2)
Seasonal Order  : (0, 1, 1) + (25)
AIC             : 10317.94
MAE             : 3.42
RMSE            : 4.35



In [26]:
# Extract model prediction
best_model_sarima = joblib.load("../models/best_sarima_model.pkl")
y_pred_sarima = best_model_sarima.get_forecast(
            steps=len(y_test),
            exog=None
        ).predicted_mean

In [24]:
# Save details of model for later use
save_model_outputs(
    model_name="SARIMA",
    y_true=y_test,
    y_pred=y_pred_sarima,
    metrics={"aic": best_sarima['aic'], 
             "mae": best_sarima['mae'], 
             "rmse":best_sarima['rmse']}
)

Saved outputs for SARIMA to model_outputs/


### 3.2 SARIMAX

In [11]:
# Fit and evaluate SARIMAX (with exogenous regressors)
DROPPED = ["avg_ground_temp"]
FEATURE_COLS = [c for c in X_unscaled_train.columns if c not in DROPPED]
exog_vars    = FEATURE_COLS
X_train_exog = X_unscaled_train[exog_vars]
X_test_exog  = X_unscaled_test[exog_vars]

best_sarimax, all_sarimax = search_best_seasonal_model(
    y_train       = y_train,
    y_test        = y_test,
    pdq_grid      = pdq_grid,
    seasonal_pdq_grid = seasonal_pdq_grid,
    seasonal_periods  = seasonal_periods,
    exog_train    = X_train_exog,
    exog_test     = X_test_exog,
    model_type    = "SARIMAX",
    log_path      = os.path.join(LOG_PATH_ROOT, "sarimax_search.log")
)

# Print SARIMAX summary
print("Best SARIMAX Configuration")
print("="*40)
print(f"Seasonal Period : {best_sarimax['seasonal_period']}")
print(f"Order           : {best_sarimax['order']}")
print(f"Seasonal Order  : {best_sarimax['seasonal_order']} + ({best_sarimax['seasonal_period']})")
print(f"AIC             : {best_sarimax['aic']:.2f}")
print(f"MAE             : {best_sarimax['mae']:.2f}")
print(f"RMSE            : {best_sarimax['rmse']:.2f}")
print("="*40)

SARIMAX grid search: 100%|██████████████████| 476/476 [3:29:19<00:00, 26.39s/it]


Best SARIMAX Configuration
Seasonal Period : 15
Order           : (0, 0, 1)
Seasonal Order  : (0, 1, 1) + (15)
AIC             : 11155.95
MAE             : 2.23
RMSE            : 3.20


In [27]:
# Extract model prediction
best_model_sarimax = joblib.load("../models/best_sarimax_model.pkl")
y_pred_sarimax = best_model_sarimax.get_forecast(
            steps=len(y_test),
            exog=X_test_exog
        ).predicted_mean

In [29]:
# Save details of model for later use
save_model_outputs(
    model_name="SARIMAX",
    y_true=y_test,
    y_pred=y_pred_sarimax,
    metrics={"aic": best_sarimax['aic'], 
             "mae": best_sarimax['mae'], 
             "rmse":best_sarimax['rmse']}
)

Saved outputs for SARIMAX to model_outputs/


---

## 4, Anomaly Detection and Classification

### 4.1 SARIMA(X) + LSTM

---

## 5. Forecasting

---

## 6. Conclusions